<a href="https://colab.research.google.com/github/AlejandroOliverosVera/blank/blob/save_state3/FS_iCSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [106]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [107]:
import random as rnd
import math
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score
from sklearn.decomposition import PCA
import sklearn.gaussian_process as gp
import tensorflow as tf
import time as tm
import gym
from gym import spaces
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch import optim
from torch.distributions.categorical import Categorical

sns.set()
DEVICE = 'cpu'

In [108]:
class ML:
  def __init__(self):
    # Leer el archivo CSV
    self.data = pd.read_csv('/content/drive/MyDrive/data.csv', header = 0)
    self.principal_components = self.find_principal_features()

  def find_principal_features(self):
    pca = PCA()
    principal_components = pca.fit_transform(self.data)
    explained_variance_ratio = pca.explained_variance_ratio_

    # Calculate the cumulative explained variance ratio
    cumulative_variance = np.cumsum(explained_variance_ratio)

    # Find the number of principal components that explain 95% of the variance
    num_components = np.argmax(cumulative_variance >= 0.95) + 1

    # Get the loadings of the first num_components
    loadings = pca.components_[:num_components]

    # Create a binary vector indicating principal features
    principal_features = np.zeros(self.data.shape[1])
    for loading in loadings:
        principal_features[np.abs(loading) >= 0.00001] = 1

    return principal_features

  def load_data(self, selected_features):
    # Usa las caracteristicas dadas por la MH
    all_features = list(self.data.columns)
    selected_column_names = [all_features[i] for i in range(len(selected_features)) if (selected_features[i] == 1)]
    return self.data[selected_column_names].values, self.data.iloc[:, -1].values.astype(float)


In [109]:
print(ML().principal_components)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

In [110]:
class MLSVM(ML):
  def train(self, selected_features):
    X, y = self.load_data(selected_features)

    # Dividir el conjunto de datos en conjuntos de entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    # Normalizar características
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Entrenar un clasificador SVM
    svm = SVC()
    svm.fit(X_train, y_train)

    # Realizar predicciones en el conjunto de prueba
    y_pred = svm.predict(X_test)

    # Calcular las métricas de evaluación
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average = 'weighted')

    return [accuracy, f1]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
class Crow:
  def __init__(self):
    self.dimension = 755 ## Depende del DSet
    self.position = []
    self.metrics = [0,0]
    self.pBest = []
    self.fBest = 0.0
    for j in range(self.dimension):
      self.position.append(rnd.randint(0,1))

  def countCols(self):
    sum = 0
    for j in range(self.dimension):
        sum += self.position[j]
    return sum

  def train(self, ml):
    self.metrics = ml.train(self.position)

  def is_better_than_pbest(self):
    if self.fitness() > self.fBest:
        self.fBest = self.fitness()
        return True
    else:
        return False

  def update_pbest(self):
        self.pBest = self.position[:]

  def isBetterThan(self, gBest):
    return self.fitness() > gBest.fBest

  def move(self, followed_crow: "Crow", AP: float, flight_length: float, ml):
    for j in range(self.dimension):
      if rnd.uniform(0, 1) >= AP:
        self.position[j] = int(self.position[j] + rnd.uniform(0, 1) * flight_length * (int(followed_crow.pBest[j]) - int(self.position[j])))
      else:
        self.position[j] = rnd.randint(0, 1)
    self.position = np.logical_or(self.position, ml.principal_components)

  def fitness(self):
    z1 = self.metrics[0] # maximizar f1 score -> [0: malo, 1: bueno]
    z2 = self.metrics[1] # maximizar acc -> [0: malo, 1: bueno]
    z3 = self.countCols() / self.dimension # minimizar cantidad de columnas que se usan -> [0: bueno, 1: malo]
    return z1 * 0.4 + z2 * 0.4 - z3 * 0.2 + 0.2

  def copy(self, other: "Crow"):
    self.position = other.position.copy()
    self.fBest = other.fBest

  #def toBinary(self, x):
    #return 1 if rnd.uniform(0, 1) <= 1 / (1 + math.exp(-x)) else 0


In [126]:
class RLCSAEnv(gym.Env):
  def __init__(self):
    self.T = 50 #iterMax
    self.nCrows = 50 #flock size
    self.AP = 0.5
    self.flightLenght = 0.75
    self.swarm = [] # array of Crow objects
    self.gCrow = Crow() #Aquí se guarda el mejor cuervo
    self.gCrow_previous = Crow()
    self.rnd = rnd
    self.sTime = None
    self.eTime = None
    high = np.array(
            [
                0,
                np.finfo(np.float32).max,
                1,
                np.finfo(np.float32).max,
            ],
            dtype=np.float32,
        )
    self.observation_space = spaces.Box(high, -high, dtype=np.float32)

    self.action_space = spaces.Discrete(4)
    self._action_to_value = {
            0: 0,
            1: 0.33,
            2: 0.66,
            3: 1,
        }
    self.ml = MLSVM()
    self.state = None

  def startTime(self):
    self.sTime = int(round(tm.time() * 1000))

  def reset(self, seed=None, options=None):
    # We need the following line to seed self.np_random
    super().reset(seed=seed)

    for i in range(self.nCrows):
      self.swarm.append(Crow())
      self.swarm[i].train(self.ml)
    self.gCrow.copy(self.swarm[0])
    for i in range(1, self.nCrows):
      if self.swarm[i].isBetterThan(self.gCrow):
        self.gCrow.copy(self.swarm[i])

    self._target_location = 1

    self.state = [self.AP, self.gCrow.fBest, 0, 1.0]

    #self.AP = self.state[0]
    #observation = self._get_obs()
    #info = self._get_info()
    #obs = np.array(self.state, dtype=np.float32), {}
    return self.state

  def step(self, action):
    self.AP = self._action_to_value[action]
    self.state[0] = self.AP
    reward = 0
    count = 0
    for i in range(self.nCrows):
      if self.swarm[i].is_better_than_pbest():
          self.swarm[i].update_pbest()
          count = count + 1
      if self.swarm[i].isBetterThan(self.gCrow):
          self.gCrow.copy(self.swarm[i])
          print("hola")
          reward = 1
    for i in range(self.nCrows):
      followedCrow = self.swarm[self.rnd.randint(0, self.nCrows - 1)]
      self.swarm[i].move(followedCrow, self.AP, self.flightLenght, self.ml)
      self.swarm[i].train(self.ml)

    self.state = [self.AP, self.gCrow.fBest, count, 1.0]
    self.log()

    terminated = True
    #print("STEP: gCrow.fBest = ", self.gCrow.fBest, "previous: ", self.gCrow_previous.fBest)
    #reward = 1 if self.gCrow.fBest > self.gCrow_previous.fBest else 0  # CALCULAR REWARD, IDEA -> GUARDAR EL BEST FITNESS DE LA STEP ANTERIOR PARA VER SI MEJORÓ O EMPEORÓ
    observation = self._get_obs()
    #info = self._get_info()
    print(np.array(self.state, dtype=np.float32), reward)
    return np.array(self.state, dtype=np.float32), reward, terminated, False, {}

  def endTime(self):
    self.eTime = int(round(tm.time() * 1000))

  def log(self):
    print(f"Mejor fitness: {self.gCrow.fBest}")

  def _get_obs(self):
    #self.state = [self.AP, self.gCrow.fBest, self.gCrow_previous.fBest, 1.0]
    return self.state
  def _get_info(self):
    return {"distance": 1-self.gCrow.fBest}  #####EN ESTE CASO GET_INFO RETORNA LA DISTANCIA ENTRE EL AGENTE Y EL TARGET





In [127]:
# Policy and value model
class ActorCriticNetwork(nn.Module):
  def __init__(self, obs_space_size, action_space_size):
    super().__init__()
    self.shared_layers = nn.Sequential(
        nn.Linear(obs_space_size, 64),
        nn.ReLU(),
        nn.Linear(64, 64),
        nn.ReLU())

    self.policy_layers = nn.Sequential(
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, action_space_size))

    self.value_layers = nn.Sequential(
        nn.Linear(64, 64),
        nn.ReLU(),
        nn.Linear(64, 1))

  def value(self, obs):
    z = self.shared_layers(obs)
    value = self.value_layers(z)
    return value

  def policy(self, obs):
    z = self.shared_layers(obs)
    policy_logits = self.policy_layers(z)
    return policy_logits

  def forward(self, obs):
    z = self.shared_layers(obs)
    policy_logits = self.policy_layers(z)
    value = self.value_layers(z)
    return policy_logits, value

In [128]:
class PPOTrainer():
  def __init__(self,
              actor_critic,
              ppo_clip_val=0.2,
              target_kl_div=0.01,
              max_policy_train_iters=80,
              value_train_iters=80,
              policy_lr=3e-4,
              value_lr=1e-2):
    self.ac = actor_critic
    self.ppo_clip_val = ppo_clip_val
    self.target_kl_div = target_kl_div
    self.max_policy_train_iters = max_policy_train_iters
    self.value_train_iters = value_train_iters

    self.n_episodes = 20
    self.print_freq = 20
    self.train_data = []
    self.reward = 0
    policy_params = list(self.ac.shared_layers.parameters()) + \
        list(self.ac.policy_layers.parameters())
    self.policy_optim = optim.Adam(policy_params, lr=policy_lr)

    value_params = list(self.ac.shared_layers.parameters()) + \
        list(self.ac.value_layers.parameters())
    self.value_optim = optim.Adam(value_params, lr=value_lr)

  def train_policy(self, obs, acts, old_log_probs, gaes):
    for _ in range(self.max_policy_train_iters):
      self.policy_optim.zero_grad()

      new_logits = self.ac.policy(obs)
      new_logits = Categorical(logits=new_logits)
      new_log_probs = new_logits.log_prob(acts)

      policy_ratio = torch.exp(new_log_probs - old_log_probs)
      clipped_ratio = policy_ratio.clamp(
          1 - self.ppo_clip_val, 1 + self.ppo_clip_val)

      clipped_loss = clipped_ratio * gaes
      full_loss = policy_ratio * gaes
      policy_loss = -torch.min(full_loss, clipped_loss).mean()

      policy_loss.backward()
      self.policy_optim.step()

      kl_div = (old_log_probs - new_log_probs).mean()
      if kl_div >= self.target_kl_div:
        break

  def train_value(self, obs, returns):
    for _ in range(self.value_train_iters):
      self.value_optim.zero_grad()

      values = self.ac.value(obs)
      value_loss = (returns - values) ** 2
      value_loss = value_loss.mean()

      value_loss.backward()
      self.value_optim.step()

  def discount_rewards(self, rewards, gamma=0.99):
    """
    Return discounted rewards based on the given rewards and gamma param.
    """
    new_rewards = [float(rewards[-1])]
    for i in reversed(range(len(rewards)-1)):
        new_rewards.append(float(rewards[i]) + gamma * new_rewards[-1])
    return np.array(new_rewards[::-1])

  def calculate_gaes(self, rewards, values, gamma=0.99, decay=0.97):
    """
    Return the General Advantage Estimates from the given rewards and values.
    Paper: https://arxiv.org/pdf/1506.02438.pdf
    """
    next_values = np.concatenate([values[1:], [0]])
    deltas = [rew + gamma * next_val - val for rew, val, next_val in zip(rewards, values, next_values)]

    gaes = [deltas[-1]]
    for i in reversed(range(len(deltas)-1)):
        gaes.append(deltas[i] + decay * gamma * gaes[-1])

    return np.array(gaes[::-1])

  def rollout(self, model, env, max_steps=1000):
    """
    Performs a single rollout.
    Returns training data in the shape (n_steps, observation_shape)
    and the cumulative reward.
    """
    ### Create data storage
    train_data = [[], [], [], [], []] # obs, act, reward, values, act_log_probs
    obs = env.reset()
    #print("obs: ", obs)
    #agent_value = obs[0]
    #print("obs en rollout: ", obs)
    ep_reward = 0
    for _ in range(max_steps):
        logits, val = model(torch.tensor([obs], dtype=torch.float32, device=DEVICE))
        act_distribution = Categorical(logits=logits)
        act = act_distribution.sample()
        act_log_prob = act_distribution.log_prob(act).item()

        act, val = act.item(), val.item()
        #print("act = ", act)
        next_obs, reward, done, _ = env.step(act)
        #print("env.AP = ", env.AP)

        for i, item in enumerate((obs, act, reward, val, act_log_prob)):
          train_data[i].append(item)

        obs = next_obs
        ep_reward += reward
        if done:
            break

    train_data = [np.asarray(x) for x in train_data]

    ### Do train data filtering
    train_data[3] = self.calculate_gaes(train_data[2], train_data[3])

    return train_data, ep_reward

  def training_loop(self, model, env):
  # Training loop
    ep_rewards = []
    for episode_idx in range(self.n_episodes):
      # Perform rollout
      train_data, reward = self.rollout(model, env)
      #train_data = train_data[0]
      ep_rewards.append(reward)
      #print("train_data[0]: ",train_data[0])
      # Shuffle
      permute_idxs = np.random.permutation(len(train_data[0]))

      # Policy data
      obs = torch.tensor(train_data[0][permute_idxs],
                        dtype=torch.float32, device=DEVICE)
      acts = torch.tensor(train_data[1][permute_idxs],
                          dtype=torch.int32, device=DEVICE)
      gaes = torch.tensor(train_data[3][permute_idxs],
                          dtype=torch.float32, device=DEVICE)
      act_log_probs = torch.tensor(train_data[4][permute_idxs],
                                  dtype=torch.float32, device=DEVICE)

      # Value data
      returns = self.discount_rewards(train_data[2])[permute_idxs]
      returns = torch.tensor(returns, dtype=torch.float32, device=DEVICE)

      # Train model
      self.train_policy(obs, acts, act_log_probs, gaes)
      self.train_value(obs, returns)

#      if (episode_idx + 1) % self.print_freq == 0:
#        print('Episode {} | Avg Reward {:.1f}'.format(
#            episode_idx + 1, np.mean(ep_rewards[-self.print_freq:])))

  def execute(self, model, env):
    self.rollout(model, env)
    self.training_loop(model, env)


In [ ]:
#####Codigo para registrar el environment como ambiente de gym

from gym.envs.registration import register

register(
    id='RLCSAEnv-v0',
    entry_point='__main__:RLCSAEnv',
    max_episode_steps=300,
)

In [130]:
env = gym.make("RLCSAEnv-v0")

#agent_observation_space = env.observation_space[0]
agent_shape = env.observation_space.shape

model = ActorCriticNetwork(env.observation_space.shape[0], env.action_space.n)
model = model.to(DEVICE)

ppo = PPOTrainer(
    model,
    policy_lr = 3e-4,
    value_lr = 1e-3,
    target_kl_div = 0.02,
    max_policy_train_iters = 40,
    value_train_iters = 40)

ppo.execute(model, env)


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:45: UserWarning: WARN: A Box observation space maximum and minimum values are equal.
  logger.warn("A Box observation space maximum and minimum values are equal.")
/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:190: UserWarning: WARN: Future gym versions will require that `Env.reset` can be passed 

hola
hola
hola
hola
hola
hola
hola
Mejor fitness: 0.8743437378557148
[ 0.66        0.87434375 50.          1.        ] 1


/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:165: UserWarning: WARN: The obs returned by the `step()` method is not within the observation space.
  logger.warn(f"{pre} is not within the observation space.")


hola
hola
hola
Mejor fitness: 0.8946124769047465
[ 0.66       0.8946125 35.         1.       ] 1
hola
Mejor fitness: 0.9009700927987865
[ 1.         0.9009701 19.         1.       ] 1
Mejor fitness: 0.9009700927987865
[0.        0.9009701 4.        1.       ] 0
hola
Mejor fitness: 0.9361804106378038
[ 0.66       0.9361804 15.         1.       ] 1
Mejor fitness: 0.9343261060020422
[ 0.         0.9343261 15.         1.       ] 0
hola
hola
Mejor fitness: 0.9669104745147521
[ 0.33       0.9669105 12.         1.       ] 1
Mejor fitness: 0.9645263685544871
[0.33       0.96452636 7.         1.        ] 0
Mejor fitness: 0.9645263685544871
[ 0.66        0.96452636 11.          1.        ] 0
Mejor fitness: 0.9645263685544871
[0.66       0.96452636 2.         1.        ] 0
Mejor fitness: 0.9645263685544871
[1.         0.96452636 4.         1.        ] 0
Mejor fitness: 0.9645263685544871
[1.         0.96452636 0.         1.        ] 0
Mejor fitness: 0.9645263685544871
[0.66       0.96452636 0.    

In [117]:
try:
  ppo.execute()
except Exception as e:
  print(f"{e} \nCaused by {e.args}")


PPOTrainer.execute() missing 2 required positional arguments: 'model' and 'env' 
Caused by ("PPOTrainer.execute() missing 2 required positional arguments: 'model' and 'env'",)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
